In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00


In [2]:
!pip install -q -U watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00
Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy       : 1.23.5
pandas      : 1.5.3
torch       : 2.0.1+cu118
transformers: 4.32.0



In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from collections import defaultdict
from textwrap import wrap

import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cpu')

In [5]:
!export CUDA_LAUNCH_BLOCKING=1

In [6]:
import pandas as pd
import numpy as np
import csv
import re
import pickle
import tqdm
from collections import Counter
import random
import os

import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train = pickle.load(open('/content/drive/MyDrive/train_new.pkl', 'rb'))
train_text = train['posts_text']
train_label = train['annotations']
    #eval = test = train
    # test set
test = pickle.load(open('/content/drive/MyDrive/test_new.pkl', 'rb'))
test_text = test['posts_text']
test_label = test['annotations']

In [ ]:
'''
Convert labels to numbers
'''

def process_data(poster, label):
    label_lookup = {'E': 1, 'I': 0, 'S': 1, 'N':0, 'T': 1, 'F': 0, 'J': 1, 'P':0}
    persona_lookup = {}
    poster_data = [{'posts': t, 'label0': label_lookup[list(label[i])[0]],
                    'label1': label_lookup[list(label[i])[1]],'label2': label_lookup[list(label[i])[2]],
                    'label3': label_lookup[list(label[i])[3]]} for i,t in enumerate(poster)]
    I,E,S,N,T,F,P,J=0,0,0,0,0,0,0,0
    for t in label:
        if 'I' in t:
            I+=1
        if 'E' in t:
            E += 1
        if 'S' in t:
            S+=1
        if 'N' in t:
            N+=1
        if 'T' in t:
            T+=1
        if 'F' in t:
            F+=1
        if 'P' in t:
            P+=1
        if 'J' in t:
            J+=1
        if t not in persona_lookup:
            persona_lookup[t] = 1
        else:
            persona_lookup[t] += 1
    return poster_data
    # process
deal_train_data = process_data(train_text, train_label)
deal_test_data = process_data(test_text, test_label)


In [9]:
test_df = pd.DataFrame(deal_test_data)
train_df = pd.DataFrame(deal_train_data)

In [10]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False)

In [11]:
#Pytorch Model
class PersonalityClassifier(nn.Module):
    def __init__(self, n_labels):
        super(PersonalityClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME,return_dict=False)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_labels*2)
    def forward(self, input_ids, attention_mask):
      _, pooled_output = self.bert(input_ids = input_ids, attention_mask=attention_mask)
      #pooled_output = outputs['pooler_output']
      pooled_output = self.drop(pooled_output)
      logits = self.out(pooled_output)
      return logits

In [12]:
# Hyperparameters
num_labels = 4
num_classes_per_label = 2
batch_size = 8
max_length = 512
learning_rate = 2e-5
epochs = 5

In [13]:
#Create labels
# Example label tensor
labels_test = torch.tensor([
    [1, 0],  # Label 1: Class 1
    [0, 1],  # Label 2: Class 2
    [1, 0],  # Label 3: Class 1
    [0, 1]   # Label 4: Class 2
])

labels0 = torch.Tensor(test_df["label0"])
labels1 = torch.Tensor(test_df["label1"])
labels2 = torch.Tensor(test_df["label2"])
labels3 = torch.Tensor(test_df["label3"])
labels_test_old = torch.stack([labels0, labels1, labels2, labels3], dim=1)
labels2 = (labels_test_old<0.5).float()
labels_test = torch.stack([labels2, labels_test_old], dim =2)
print(labels_test[0])
print(labels_test.shape)

labels0 = torch.Tensor(train_df["label0"])
labels1 = torch.Tensor(train_df["label1"])
labels2 = torch.Tensor(train_df["label2"])
labels3 = torch.Tensor(train_df["label3"])
labels_train_old = torch.stack([labels0, labels1, labels2, labels3], dim=1)
labels2 = (labels_train_old<0.5).float()
labels_train = torch.stack([labels2, labels_train_old], dim =2)
print(labels_train[0])
print(labels_train_old.shape)
print(labels_train.shape)

tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]])
torch.Size([1814, 4, 2])
tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
torch.Size([7253, 4])
torch.Size([7253, 4, 2])


In [14]:
class CustomDataset(Dataset):
    def __init__(self, posts, labels, labels_old, tokenizer, max_len):
        self.posts = posts
        self.labels = labels
        self.labels_old = labels_old
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.posts)

    def __getitem__(self, item):
        posts = str(self.posts[item])
        label = self.labels[item]
        label_old = self.labels_old[item]
        encoding = self.tokenizer.encode_plus(
          posts,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation=True,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt'
    )
        input_ids= encoding['input_ids'].flatten()
        attention_mask= encoding['attention_mask'].flatten()
        return {
           'input_ids': torch.tensor(input_ids),
           'attention_mask': torch.tensor(attention_mask),
          'labels' : label,
          'labels_old' : label_old
        }

In [15]:
train_text = train_df["posts"].tolist()
test_text = test_df["posts"].tolist()

train_dataset = CustomDataset(train_text, labels_train, labels_train_old, tokenizer, max_length)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_text, labels_test, labels_test_old, tokenizer, max_length)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'labels_old'])

In [ ]:
print(len(train_data_loader))

907


In [ ]:
yhat = model(data['input_ids'].to(device), data['attention_mask'].to(device))

In [ ]:
!pip install torchviz
from torchviz import make_dot

make_dot(yhat.mean(), params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

In [ ]:
print(len(data['input_ids']))
print(len(data['labels_old']))

In [ ]:
print(type(data['input_ids']))
print(type(data['attention_mask']))
print(type(data['labels']))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['labels'].shape)
print(data['labels'][1])

torch.Size([8, 1, 512])
torch.Size([8, 1, 512])
torch.Size([8, 4, 2])
tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])


In [16]:
model = PersonalityClassifier(num_labels)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
print(model)

PersonalityClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [17]:
from sklearn.metrics import accuracy_score
def train_model(model,
                  epochs,
                  train_data_loader,
                  optimizer,
                  device):
  for epoch in range(epochs):
      total_loss = 0.0
      all_true_labels = []
      all_pred_labels = []

      for dl in train_data_loader:
          input_ids = dl['input_ids'].to(device)
          attention_mask =  dl['attention_mask'].to(device)
          labels = dl['labels'].to(device)
          labels_old = dl['labels_old'].to(device)
          optimizer.zero_grad()
          logits = model(input_ids=input_ids, attention_mask=attention_mask)
          logits = logits.view(-1, num_labels, num_classes_per_label)
          #labels = labels.view(-1, num_labels)
          #print(labels)
          loss = criterion(logits, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

          _, predicted = torch.max(logits, dim=2)
          all_true_labels.append(labels_old)
          all_pred_labels.append(predicted)
          print(len(all_true_labels))
          print(len(all_pred_labels))

      avg_loss = total_loss / len(train_data_loader)
      true_labels = torch.cat(all_true_labels, dim= 0)
      pred_labels = torch.cat(all_pred_labels, dim =0)
  return avg_loss, true_labels, pred_labels


print("Training finished.")

Training finished.


In [ ]:
def test_model(model, test_data_loader, device):
  model = model.eval()
  total_loss = 0.0
  all_true_labels = []
  all_pred_labels = []
  with torch.no_grad():
        for dl in test_data_loader:
          input_ids = dl['input_ids'].to(device)
          attention_mask =  dl['attention_mask'].to(device)
          labels = dl['labels'].to(device)
          labels_old = dl['labels_old'].to(device)
          logits = model(input_ids=input_ids, attention_mask=attention_mask)
          logits = logits.view(-1, num_labels, num_classes_per_label)
          loss = criterion(logits, labels)
          total_loss += loss.item()
          _, predicted = torch.max(logits, dim=2)
          all_true_labels.append(labels_old)
          all_pred_labels.append(predicted)
        avg_loss = total_loss / len(test_data_loader)
        print(len(all_true_labels))
        print(len(all_pred_labels))
        true_labels = torch.cat(all_true_labels, dim= 0)
        pred_labels = torch.cat(all_pred_labels, dim =0)

  return avg_loss, true_labels, pred_labels

In [ ]:
epochs = 4
avg_loss, true_labels, pred_labels = train_model(model, epochs, train_data_loader, optimizer, device)
print(len(true_labels.view(-1).cpu().numpy()[:29012]))
acc= accuracy_score(true_labels.view(-1).cpu().numpy()[:29012], pred_labels.view(-1).cpu().numpy())
print('loss ',avg_loss)
print(acc)



In [ ]:
print(len(true_labels.view(-1).cpu().numpy()[:29012]))
acc= accuracy_score(true_labels.view(-1).cpu().numpy()[:29012], pred_labels.view(-1).cpu().numpy())
print('loss ',avg_loss)
print(acc)

29012
loss  2.58087420148124
0.6847511374603612


In [ ]:
torch.save(model.state_dict(), 'best_model_state.bin')

In [ ]:
epochs = 1
avg_loss, true_labels, pred_labels = test_model(model, test_data_loader, device)
print(len(true_labels.view(-1).cpu().numpy()[:7456]))
acc= accuracy_score(true_labels.view(-1).cpu().numpy()[:7456], pred_labels.view(-1).cpu().numpy())
print('loss ',avg_loss)
print(acc)
